In [6]:
#******************************************************************************************************************************************
#
#                                                     Binary Convolutional Codes                                                          *
#                                                                                                                                         *
#     This code implements a Binary Convolutional Encoder and Decoder,which are essential components in digital communication systems,    *
#          particularly in error control coding schemes. This code implements a basic binary convolutional encoding and                   *
#                decoding scheme with error correction using matrix operations. The Viterbi algorithm is used for                         *
#                                  decoding to find the most likely path through the trellis.                                             *
#
#******************************************************************************************************************************************

In [7]:
import numpy as np

In [8]:
class BinaryConvolutionalEncoder:
    '''
       This class is responsible for encoding input binary data using a generator matrix.
       The encode method takes input data, shifts it through a shift register,
       and then performs matrix multiplication with the generator matrix to produce encoded data.
    '''

    def __init__(self, generator_matrix):
        self.generator_matrix = generator_matrix

    def encode(self, data):                                               # Define a method called encode, which takes data as input
        data_length = len(data)                                           # Calculate the length of the input data
        state = np.zeros(self.generator_matrix.shape[1], dtype=int)       # Initialize a state array of zeros
        encoded_data = []                                                 # Initialize an empty list to store the encoded data
        for i in range(data_length):                                      # Iterate over the input data
            bit = int(data[i])                                            # Get the current bit from the input data
            state = np.roll(state, 1)                                     # Rotate the state array by 1 position to the right
            state[0] = bit                                                # Update the first element of the state array with the current bit
            encoded_bit = np.dot(self.generator_matrix, state) % 2        # Calculate the encoded bit using matrix multiplication and modulo 2 operation
            encoded_data.extend(encoded_bit)                              # Add the encoded bit to the encoded data list

        return encoded_data

In [9]:
class BinaryConvolutionalDecoder:
    '''
       This class implements a basic decoder for convolutionally encoded data.
       The decode method takes received data, performs syndrome calculation using the parity check matrix, and corrects errors if any.
    '''


    def __init__(self, parity_check_matrix):                              # Constructor method to initialize the decoder with a parity check matrix
        self.parity_check_matrix = parity_check_matrix                    # Store the parity check matrix in the decoder object

    def decode(self, received_data):                                      # Method to decode received binary data
        data_length = len(received_data)                                  # Get the length of the received data
        state = np.zeros(self.parity_check_matrix.shape[1], dtype=int)    # Initialize a state array of zeros
        decoded_data = []                                                 # Initialize an empty list to store the decoded data
        for i in range(data_length):                                      # Iterate over the received data
            bit = int(received_data[i])                                   # Get the current bit from the received data
            state = np.roll(state, 1)                                     # Rotate the state array by 1 position to the right
            state[0] = bit                                                # Update the first element of the state array with the current bit
            syndrome = np.dot(self.parity_check_matrix, state) % 2        # Calculate the syndrome using matrix multiplication and modulo 2 operation

            if np.count_nonzero(syndrome) == 0:                           # If no errors detected in the syndrome
                decoded_data.append(bit)                                  # Append the current bit to the decoded data
            else:                                                         # If errors are detected in the syndrome, # Error detected, flipping the bit
                bit = 1 - bit                                             # Flip the current bit
                decoded_data.append(bit)                                  # Append the corrected bit to the decoded data

        return decoded_data


    def viterbi_decode(self, received_data):
        '''
          The viterbi_decode method implements the Viterbi algorithm for convolutional decoding.
          It iterates through the received data, calculating branch metrics for each state transition and updating the trellis accordingly.
          Then, it backtraces to find the most likely path through the trellis, which represents the decoded data.
        '''


        trellis = np.zeros((len(received_data) + 1, 2), dtype=int)       # Trellis initialization
        decoded_data = []
        for i in range(len(received_data)):
            bit = int(received_data[i])
            branch_metric = np.zeros(2)                                  # Branch metrics for each state transition
            for state in range(2):
                next_state = (state + bit) % 2                           # Transition to the next state
                branch_metric[next_state] = np.count_nonzero(self.parity_check_matrix[next_state] ^ bit)
            trellis[i + 1] = trellis[i] + branch_metric                  # Update the trellis with branch metrics

        # Backtrace to find the most likely path
        state = np.argmin(trellis[-1])                                   # Determine the ending state
        for i in range(len(received_data), 0, -1):
            state = (state + int(received_data[i - 1])) % 2              # Update the state based on received data
            decoded_data.append(state)

        decoded_data.reverse()                                           # Reverse the decoded data since we traversed backward
        return decoded_data

In [10]:
def  banner():
    print("""

***********************************************************************************************************************************
*                                                                                                                                 *
*                                              Binary Convolutional Codes                                                         *
*                                                                                                                                 *
*             This program demonstrates a binary convolutional encoder and Viterbi decoder algorithm implementation.              *
*         The encoder takes input binary data and encodes it using a generator matrix. The decoder then decodes received          *
*       data using a parity check matrix. You will be prompted to enter the dimensions and elements of the generator matrix       *
*           and the parity check matrix. After that, you can input data to be encoded and received data to be decoded.            *
*                                                                                                                                 *
***********************************************************************************************************************************
        """)

def main():
    banner()
    generator_rows = int(input("\n---> Enter the number of in the generator matrix: "))
    generator_cols = int(input("---> Enter the number of columns in the generator matrix: "))
    generator_matrix = np.zeros((generator_rows, generator_cols), dtype=int)
    print("---> Enter the elements of the generator matrix:")
    for i in range(generator_rows):
        for j in range(generator_cols):
            generator_matrix[i, j] = int(input(f"    - Enter element ({i}, {j}): "))
    print("**************************************************************************************************************************\n")
    parity_rows = int(input("---> Enter the number of rows in the parity check matrix: "))
    parity_cols = int(input("---> Enter the number of columns in the parity check matrix: "))
    parity_check_matrix = np.zeros((parity_rows, parity_cols), dtype=int)
    print("---> Enter the elements of the parity check matrix:")
    for i in range(parity_rows):
        for j in range(parity_cols):
            parity_check_matrix[i, j] = int(input(f"    - Enter element ({i}, {j}): "))
    print("**************************************************************************************************************************\n")
    encoder = BinaryConvolutionalEncoder(generator_matrix)
    decoder = BinaryConvolutionalDecoder(parity_check_matrix)
    data = input("---> Enter the data to be encoded: ")
    encoded_data = encoder.encode(data)
    print("    **** Encoded data:", encoded_data)
    received_data = input("---> Enter the received data to be decoded: ")
    decoded_data = decoder.viterbi_decode(received_data)
    print("    **** Decoded data:", decoded_data)
    print("**************************************************************************************************************************")


if __name__ == "__main__":
    main()


                
***********************************************************************************************************************************
*                                                                                                                                 *
*                                              Binary Convolutional Codes                                                         *
*                                                                                                                                 *
*             This program demonstrates a binary convolutional encoder and Viterbi decoder algorithm implementation.              *
*         The encoder takes input binary data and encodes it using a generator matrix. The decoder then decodes received          *
*       data using a parity check matrix. You will be prompted to enter the dimensions and elements of the generator matrix       *
*           and the parity check matrix. After that, you c